In [5]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from libs.common import *
from config.config import *

c:\Users\PC\Data\ads_fraud_detection


# Lựa chọn đặc trưng

### (1) Tạo thư mục lưu trữ (nếu chưa có)

In [6]:
save_dir=f"{exps_dir}/exp_prj_fake_jobs_postings"

os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [10]:
from scipy import sparse
x_train= sparse.load_npz(f'{save_dir}/x_train.npz')
y_train=pd.read_csv(f'{save_dir}/y_train.csv')
x_test= sparse.load_npz(f'{save_dir}/x_test.npz')
y_test=pd.read_clipboard(f'{save_dir}/y_test.csv')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(22318, 1667122) (22318, 1) (5575, 1667122) (0, 1)


### (2) Lựa chọn các đặc trưng

In [11]:
x_train

<22318x1667122 sparse matrix of type '<class 'numpy.int64'>'
	with 10386286 stored elements in Compressed Sparse Row format>

In [12]:
def modelcheck_info(model,name):
    print(name)
    try:
        coefficients = model.coef_.flatten()
    except:
        coefficients = model.feature_importances_.flatten()
    feature_importance = pd.DataFrame({
        'Feature': x_train.columns,
        'Coefficient': coefficients
    })

    print('bi loai bo',feature_importance[feature_importance['Coefficient'] == 0])
    feature_importance = feature_importance[feature_importance['Coefficient'] != 0]
    feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)

    print("Xếp hạng các đặc trưng theo mức độ quan trọng:")
    print(feature_importance[['Feature', 'Coefficient']])

In [13]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
names=['LR','LDA','DTC']
feature_SelectFromModel={}
for model,name in zip(models,names):
    check = model.fit(x_train, y_train)
    selector = SelectFromModel(check, prefit=True)

    selected_features = x_train.columns[(selector.get_support())]
    modelcheck_info(model,name)
    print("Các đặc trưng được chọn:")
    feature_SelectFromModel[name]=list(selected_features)
    print(selected_features)

AttributeError: columns not found

### Giải thích:

Phương pháp **SelectFromModel** từ thư viện sklearn.feature_selection. Phương pháp này hoạt động bằng cách sử dụng một mô hình học máy để đánh giá độ quan trọng của từng đặc trưng và sau đó chọn những đặc trưng quan trọng nhất.

   Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
  
       #### 1. LogisticRegression:
       Các đặc trưng được chọn:
* Index(['age', 'policy_csl', 'insured_education_level', 'insured_occupation',
       'incident_severity', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'property_damage', 'witnesses',
       'property_claim', 'vehicle_claim', 'auto_make'],
      dtype='object')
      
   #### 2. LinearDiscriminantAnalysis:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip', 'insured_sex',
       'incident_severity', 'incident_hour_of_the_day', 'property_damage',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')
      
   #### 3. DecisionTreeClassifier:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip',
       'insured_occupation', 'insured_hobbies', 'incident_severity',
       'incident_location', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'month_policy_bind_date', 'day_incident_date'],
      dtype='object')

In [ ]:
def feature_selection_inRFE(model,name):
    rfe=RFE(estimator=model, n_features_to_select=10)
    fit=rfe.fit(x_train, y_train)
#     print("Num Features: %d"% fit.n_features_)
#     print("Selected Features: %s" % fit.support_)
#     print("Features Ranking: %s" %fit.ranking_)
#     for index, value in enumerate(fit.ranking_):
#         print(f"{value}: {x_train.columns[index]}")
    sorted_columns = [x for _, x in sorted(zip(fit.ranking_, x_train.columns))]
    sort=sorted(fit.ranking_)

#     for i,j in zip(sort,sorted_columns):
#         print("Columns sorted by RFE ranking",i,":",j)

    return [j for i,j in zip(sort,sorted_columns) if i==1]

In [ ]:
models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LGBMClassifier(),  
    XGBClassifier(use_label_encoder=False)  
]
feature_RFE={}

names = ['LR', 'LDA', 'DTC', 'RD', 'LGBM', 'XGB']
for model,name in zip(models,names):
    print("RFE")
    print(f"Kiểm tra mô hình {name}")
    model.fit(x_train,y_train)
    print(feature_selection_inRFE(model,name))
    feature_RFE[name]=feature_selection_inRFE(model,name)

RFE
Kiểm tra mô hình LR
['Age', 'AgeOfPolicyHolder', 'AgentType', 'BasePolicy', 'Fault', 'MaritalStatus', 'PoliceReportFiled', 'PolicyType', 'VehicleCategory', 'WitnessPresent']
RFE
Kiểm tra mô hình LDA
['AccidentArea', 'Age', 'AgeOfPolicyHolder', 'BasePolicy', 'Days_Policy_Claim', 'Fault', 'MaritalStatus', 'NumberOfCars', 'PoliceReportFiled', 'VehicleCategory']
RFE
Kiểm tra mô hình DTC
['Age', 'AgeOfVehicle', 'BasePolicy', 'Fault', 'Make', 'PastNumberOfClaims', 'PolicyNumber', 'RepNumber', 'week_claimed', 'week_incident']
RFE
Kiểm tra mô hình RD
['Age', 'AgeOfVehicle', 'BasePolicy', 'Fault', 'Make', 'PolicyNumber', 'PolicyType', 'VehicleCategory', 'week_claimed', 'week_incident']


### Giải thích:
**Recursive Feature Elimination (RFE)** là một phương pháp trong lựa chọn đặc trưng (feature selection) được sử dụng trong học máy để giảm số lượng đặc trưng của mô hình mà vẫn giữ được hiệu suất tốt.

Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
   * **'RD'**: RandomForestClassifier

In [ ]:
special_models=[KNeighborsClassifier(),GaussianNB(),MLPClassifier(),GradientBoostingClassifier(),MultinomialNB(),SVC()]
names=['KNC','GNB','NN','BG','NB','SVM']
def feature_selection_inKBest(model, name):
    k_best = SelectKBest(score_func=f_classif, k=11)
    fit = k_best.fit(x_train, y_train)
    selected_features = fit.get_support()
    selected_column_names = x_train.columns[selected_features]
    return selected_column_names
feature_Kbest={}
for model,name in zip(special_models,names):
    print(f'Lua chon dac trung {name}')
    print(feature_selection_inKBest(model, name))
    feature_Kbest[name]=list(feature_selection_inKBest(model, name))

Lua chon dac trung KNC
Index(['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], dtype='object')
Lua chon dac trung GNB
Index(['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], dtype='object')
Lua chon dac trung NN
Index(['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], dtype='object')
Lua chon dac trung BG
Index(['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], dtype='object')
Lua chon dac trung NB
Index(['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 

### Giải thích:
Phương pháp **SelectKBest** từ thư viện sklearn.feature_selection để lựa chọn đặc trưng dựa trên kỹ thuật kiểm định thống kê.
 
Các model được dùng trong phương pháp này:
   * **'KNC'**: KNeighborsClassifier
   * **'GNB'**: GaussianNB
   * **'NN'**: MLPClassifier
   * **'BG'**: GradientBoostingClassifier
   * **'NB'**: MultinomialNB
   * **'SVM'**: SVC

In [ ]:
print(feature_Kbest)
print(feature_SelectFromModel)
print(feature_RFE)

{'KNC': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'GNB': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'NN': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'BG': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'NB': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'SVM': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatu

In [ ]:
feature_model_selection={**feature_RFE,**feature_Kbest}
np.savez(f'{save_dir}/feature_model_selection.npz', feature_model_selection = feature_model_selection)
feature_model_selection

{'LR': ['Age',
  'AgeOfPolicyHolder',
  'AgentType',
  'BasePolicy',
  'Fault',
  'MaritalStatus',
  'PoliceReportFiled',
  'PolicyType',
  'VehicleCategory',
  'WitnessPresent'],
 'LDA': ['AccidentArea',
  'Age',
  'AgeOfPolicyHolder',
  'BasePolicy',
  'Days_Policy_Claim',
  'Fault',
  'MaritalStatus',
  'NumberOfCars',
  'PoliceReportFiled',
  'VehicleCategory'],
 'DTC': ['Age',
  'AgeOfVehicle',
  'BasePolicy',
  'Fault',
  'Make',
  'PastNumberOfClaims',
  'PolicyNumber',
  'RepNumber',
  'week_claimed',
  'week_incident'],
 'RD': ['Age',
  'AgeOfVehicle',
  'BasePolicy',
  'Fault',
  'Make',
  'PolicyNumber',
  'PolicyType',
  'VehicleCategory',
  'week_claimed',
  'week_incident'],
 'KNC': ['DayOfWeek',
  'AccidentArea',
  'WeekOfMonthClaimed',
  'MaritalStatus',
  'Fault',
  'PolicyType',
  'VehicleCategory',
  'DriverRating',
  'AgeOfVehicle',
  'AgeOfPolicyHolder',
  'BasePolicy'],
 'GNB': ['DayOfWeek',
  'AccidentArea',
  'WeekOfMonthClaimed',
  'MaritalStatus',
  'Fault',
 

* gom nhóm thông tin về các đặc trưng được chọn từ hai phương pháp lựa chọn đặc trưng khác nhau (RFE và SelectKBest) vào từ điển **feature_model_selection**.

In [ ]:
dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']

array({'LR': ['Age', 'AgeOfPolicyHolder', 'AgentType', 'BasePolicy', 'Fault', 'MaritalStatus', 'PoliceReportFiled', 'PolicyType', 'VehicleCategory', 'WitnessPresent'], 'LDA': ['AccidentArea', 'Age', 'AgeOfPolicyHolder', 'BasePolicy', 'Days_Policy_Claim', 'Fault', 'MaritalStatus', 'NumberOfCars', 'PoliceReportFiled', 'VehicleCategory'], 'DTC': ['Age', 'AgeOfVehicle', 'BasePolicy', 'Fault', 'Make', 'PastNumberOfClaims', 'PolicyNumber', 'RepNumber', 'week_claimed', 'week_incident'], 'RD': ['Age', 'AgeOfVehicle', 'BasePolicy', 'Fault', 'Make', 'PolicyNumber', 'PolicyType', 'VehicleCategory', 'week_claimed', 'week_incident'], 'KNC': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'GNB': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'],